In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import pywt
import os

C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
path_dir_1= 'C:/Users/SDML/1GAN/Normal'
path_dir_2 = 'C:/Users/SDML/1GAN/Misalign'

In [3]:
file_list_Normal = os.listdir(path_dir_1)
file_list_Misalign = os.listdir(path_dir_2)

In [4]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [5]:
NoOfCondition = 1  # RPM 따로따로
# NoOfData = 500
NoOfSensor = 3
NoOfFeature = 10


# Feature Extraction

### Time Domain 특징값 추출 (10 features * 2 sensors = 20개씩)

In [6]:
len(file_list_Misalign)

382

In [7]:
# 특징데이터 크기 지정
TimeFeature_Normal   = np.zeros((NoOfSensor*NoOfFeature , len(file_list_Normal)))
TimeFeature_Misalign   = np.zeros((NoOfSensor*NoOfFeature , len(file_list_Misalign)))

for i in range(len(file_list_Normal)):
    
    temp_path1 = path_dir_1 + '/' + file_list_Normal[i]  # Normal 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1).iloc[:,3:] # 임시 Normal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[10*j+0, i] = np.max(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+1, i] = np.min(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+2, i] = np.mean(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+3, i] = rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+4, i] = np.var(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+5, i] = sp.skew(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+7, i] = np.max(temp_data1.iloc[:,j])/rms(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+8, i] = rms(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        TimeFeature_Normal[10*j+9, i] = np.max(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        
        
print(TimeFeature_Normal.shape)

(30, 628)


In [8]:
for i in range(len(file_list_Misalign)):
    
    temp_path1 = path_dir_2 + '/' + file_list_Misalign[i]  # Misalign 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1).iloc[:,3:] # 임시 Misalign 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Misalign Time Domain Feature
        TimeFeature_Misalign[10*j+0, i] = np.max(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+1, i] = np.min(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+2, i] = np.mean(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+3, i] = rms(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+4, i] = np.var(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+5, i] = sp.skew(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+7, i] = np.max(temp_data1.iloc[:,j])/rms(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+8, i] = rms(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        TimeFeature_Misalign[10*j+9, i] = np.max(temp_data1.iloc[:,j])/np.mean(temp_data1.iloc[:,j])
        

In [9]:
TimeFeature = np.concatenate([TimeFeature_Normal,TimeFeature_Misalign] , axis=1)
TimeFeature.shape

(30, 1010)

### Frequency Domain 특징값 추출 (10 features * 8 wavelet levels * 2 sensors = 160개씩)

In [10]:
# Wavelet options
MotherWavelet = pywt.Wavelet('haar')   # Mother wavelet (모함수) 지정
Level   = 8                            # Wavelet 분해 레벨 지정
select  = 8                            # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [11]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , len(file_list_Normal)))

for i in range(len(file_list_Normal)):
    
    # 데이터 불러오기
    temp_path1 = path_dir_1 + '/' + file_list_Normal[i]  # Normal 데이터 파일 경로
    temp_data1 = np.array(pd.read_csv(temp_path1).iloc[:,3:6]) # 임시 Normal 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            
            # 영상과 다름 주의! : 영상에서는 (i-1)이지만 i 가 맞는 코드
            # Normal Frequency Domain Feature
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
           



In [12]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Misalign   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , len(file_list_Misalign)))

for i in range(len(file_list_Misalign)):
    
    # 데이터 불러오기
    temp_path1 = path_dir_2 + '/' + file_list_Misalign[i]  # Misalign 데이터 파일 경로
    temp_data1 = np.array(pd.read_csv(temp_path1).iloc[:,3:6]) # 임시 Misalign 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in range(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            
            # 영상과 다름 주의! : 영상에서는 (i-1)이지만 i 가 맞는 코드
            # Misalign Frequency Domain Feature
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+0 , i] = np.max(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+1 , i] = np.min(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+2 , i] = np.mean(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+3 , i] = np.var(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+4 , i] = rms(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+5 , i] = sp.skew(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+6 , i] = sp.kurtosis(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+7 , i] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+8 , i] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeature_Misalign[NoOfFeature*j*select+k*NoOfFeature+9 , i] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
           



In [13]:
FreqFeature = np.concatenate([FreqFeature_Normal,FreqFeature_Misalign] , axis=1)
FreqFeature.shape

(240, 1010)

# Feature 통합

In [14]:
Features = np.concatenate([TimeFeature,FreqFeature] , axis=0)
FeatureData = pd.DataFrame(Features)
print("Feature Data Size :", Features.shape)
path = './ProcessedData/[Real]FeatureData'   # path = '파일 경로/저장할 파일 이름'
FeatureData.to_csv(path, sep=',', header=None , index=None)

Feature Data Size : (270, 1010)


### .
### 
### .

In [15]:
import seaborn           as sb
import scipy.stats       as sp
import matplotlib.pyplot as plt
from scipy.stats import f_oneway

특징 데이터 로드

In [16]:
s8 = "FeatureData = pd.read_csv('./ProcessedData/[Real]FeatureData', sep=',', header=None)"
exec(s8)
print(FeatureData.shape)

(270, 1010)


In [17]:
Normal_FeatureData   = FeatureData.iloc[:,:len(file_list_Normal)]
Misalign_FeatureData = FeatureData.iloc[:,len(file_list_Normal):len(file_list_Normal)+len(file_list_Misalign)]

print(Normal_FeatureData.shape)
print(Misalign_FeatureData.shape)

(270, 628)
(270, 382)


#### P-value 계산

In [18]:
FeatureNameArray = []

SensorName = ["Current", "Voltage","Acc"]
FeatureName = ["max", "min", "mean","rms", "var", "skew", "kurtosis", "crest", "impulse", "shape"]

for i in range(3) :
    for j in range(10) :
        newname = "Time_" + SensorName[i] + "_" + FeatureName[j]
        FeatureNameArray.append(newname)

for i in range(3) :
    for j in range(8) :
        for k in range(10) :
            s = "newname = 'WT_' + SensorName[i] + '_d%d_' + FeatureName[k]"%(j+1)
            exec(s)
            FeatureNameArray.append(newname)
    
FeatureNameArray = pd.DataFrame(FeatureNameArray)
FeatureNameArray

,0
0,Time_Current_max
1,Time_Current_min
2,Time_Current_mean
3,Time_Current_rms
4,Time_Current_var
...,...
265,WT_Acc_d8_skew
266,WT_Acc_d8_kurtosis
267,WT_Acc_d8_crest
268,WT_Acc_d8_impulse


In [19]:
Normal_FeatureData

,0,1,2,3,4,5,6,7,8,9,...,618,619,620,621,622,623,624,625,626,627
0,5.134225,5.160453,5.217757,5.136864,5.149439,5.153120,5.158654,5.155789,5.116391,5.114415,...,5.170418,5.166774,5.135882,5.130690,5.156307,5.190843,5.176881,5.157683,5.135710,5.129123
1,-5.174488,-5.197548,-5.246090,-5.166645,-5.177817,-5.192472,-5.197657,-5.176036,-5.158000,-5.145766,...,-5.242558,-5.193615,-5.171443,-5.176482,-5.188843,-5.187913,-5.205402,-5.192910,-5.160069,-5.163061
2,-0.021757,-0.021086,-0.020869,-0.021364,-0.017953,-0.021804,-0.019838,-0.017278,-0.020363,-0.022055,...,-0.023628,-0.022805,-0.023236,-0.023362,-0.022355,-0.020279,-0.021512,-0.021721,-0.021610,-0.025019
3,2.710191,2.710649,2.706833,2.705323,2.702243,2.708631,2.711088,2.703123,2.699435,2.701425,...,2.707609,2.698283,2.699532,2.701642,2.692496,2.700987,2.707201,2.700764,2.707651,2.694406
4,7.344661,7.347173,7.326512,7.318315,7.301795,7.336206,7.349603,7.306575,7.286533,7.297208,...,7.330587,7.280213,7.286932,7.298326,7.249036,7.294922,7.328476,7.293654,7.330906,7.259197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,0.878204,0.690012,0.938742,0.908793,1.300501,0.889881,1.033361,1.160249,1.137002,0.915720,...,0.845818,1.172528,1.151129,0.895851,0.937663,0.952024,0.983867,0.834209,0.995568,1.022149
266,-0.316316,-0.315369,-0.152751,0.219381,0.863836,-0.237469,-0.139801,0.113240,0.716479,0.249908,...,-0.279404,0.127165,0.363117,-0.230079,-0.020998,-0.203635,0.015090,-0.300405,-0.204917,-0.002176
267,2.237681,2.228348,2.159332,2.310891,2.567706,2.216535,2.274008,2.353966,2.497364,2.303884,...,2.185523,2.226798,2.336196,2.190978,2.304118,2.173205,2.359958,2.236995,2.153012,2.266461
268,11.698839,27.521484,28.049921,-55.322059,10.103295,17.448659,5.226498,9.074965,19.313218,-65.452906,...,12.173795,16.931661,26.288777,13.450861,9.862516,8.207896,6.707789,7.884754,10.718287,16.529556


In [20]:
Misalign_FeatureData

,628,629,630,631,632,633,634,635,636,637,...,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009
0,5.195401,5.105005,5.117804,5.110070,5.161499,5.122798,5.245968,5.196249,5.118442,5.140696,...,5.175224,5.242781,5.161263,5.112533,5.181388,5.173496,5.135968,5.111034,5.088049,5.253526
1,-5.183710,-5.157253,-5.141590,-5.145414,-5.166675,-5.147397,-5.267144,-5.200776,-5.116877,-5.142452,...,-5.195475,-5.236211,-5.157010,-5.142167,-5.166491,-5.188588,-5.129882,-5.125046,-5.116505,-5.276281
2,-0.017223,-0.018112,-0.018696,-0.020184,-0.016692,-0.017244,-0.018867,-0.019409,-0.018557,-0.018451,...,-0.019514,-0.017162,-0.018062,-0.018377,-0.015621,-0.016184,-0.015234,-0.014149,-0.016353,-0.021355
3,2.700462,2.686376,2.675818,2.663979,2.677200,2.680044,2.673608,2.668402,2.667435,2.678781,...,2.673168,2.673626,2.699137,2.683350,2.672704,2.678978,2.670254,2.663046,2.673898,2.673147
4,7.292201,7.216289,7.159653,7.096378,7.167121,7.182341,7.147822,7.119990,7.114864,7.175528,...,7.145445,7.147983,7.285014,7.200028,7.143101,7.176662,7.130024,7.091612,7.149462,7.145257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,1.359770,1.297649,1.246060,1.244116,1.146326,1.003464,1.310632,1.325185,1.337255,1.352582,...,1.202535,1.326343,1.316167,1.287252,1.101660,1.189734,1.241686,1.198559,1.272923,1.235325
266,0.539906,0.209706,0.019154,0.011862,0.527493,0.187161,0.339897,0.234993,0.230698,0.457746,...,0.111738,0.320404,0.540622,0.227517,-0.034866,0.100484,0.095432,-0.141940,0.036600,0.186673
267,2.493746,2.149878,2.161422,2.198404,2.529720,2.451494,2.377114,2.234479,2.162029,2.412725,...,2.358668,2.291400,2.477355,2.371440,2.075708,2.149030,2.223849,2.166218,2.185677,2.354192
268,5.423532,9.789665,4.936776,5.741266,4.486685,4.232097,6.186341,6.377162,9.817499,6.195023,...,5.471297,11.818733,5.755076,3.531049,7.885843,9.109792,8.363635,5.558378,5.737753,5.988971


In [21]:
NoOfFeature = FeatureNameArray.shape[0] # 추출된 Feature 갯수

P_value = np.zeros((NoOfFeature , 2))

# 특징값 각각 T-검정 수행
for i in np.arange(NoOfFeature):
    F , p = f_oneway(Normal_FeatureData.iloc[i,:],Misalign_FeatureData.iloc[i,:])
    P_value[i,0] = i          # Feature Index
    P_value[i,1] = p  # P값 (P-value)
    
P_value      = pd.DataFrame(P_value)

P_value_DataOnly = np.zeros((P_value.shape[0],1))
P_value_DataOnly[:,0] = P_value.iloc[:,1].values
P_value_NumOnly = np.zeros((P_value.shape[0],1))
P_value_NumOnly[:,0] = P_value.iloc[:,0].values
P_value = pd.DataFrame(np.concatenate([P_value_NumOnly,FeatureNameArray, P_value_DataOnly] , axis=1))
pd.set_option('display.max_row',270)
P_value

,0,1,2
0,0.0,Time_Current_max,0.000008
1,1.0,Time_Current_min,0.0
2,2.0,Time_Current_mean,0.0
3,3.0,Time_Current_rms,0.0
4,4.0,Time_Current_var,0.0
5,5.0,Time_Current_skew,0.0
6,6.0,Time_Current_kurtosis,0.0
7,7.0,Time_Current_crest,0.0
8,8.0,Time_Current_impulse,0.0
9,9.0,Time_Current_shape,0.0


In [22]:
P_value_Rank = P_value.sort_values([2],ascending=True)  # P-value 기준 오름차순 정렬
P_value_Rank

,0,1,2
234,234.0,WT_Acc_d5_var,0.0
240,240.0,WT_Acc_d6_max,0.0
244,244.0,WT_Acc_d6_var,0.0
243,243.0,WT_Acc_d6_rms,0.0
233,233.0,WT_Acc_d5_rms,0.0
250,250.0,WT_Acc_d7_max,0.0
180,180.0,WT_Voltage_d8_max,0.0
3,3.0,Time_Current_rms,0.0
4,4.0,Time_Current_var,0.0
254,254.0,WT_Acc_d7_var,0.0


In [23]:
P_value_Rank.iloc[0,2] < P_value_Rank.iloc[2,2] 

False

In [24]:
# 특징값 저장
s9  = "path1 = './ProcessedData/P_value'"       ;exec(s9)
s10 = "path2 = './ProcessedData/P_value_Rank'"  ;exec(s10)

P_value.to_csv(path1, sep=',', header=None, index=None)
P_value_Rank.to_csv(path2, sep=',', header=None, index=None)

In [25]:
Rank = 30

Normal   = np.zeros((Rank,len(file_list_Normal)))
Misalign = np.zeros((Rank,len(file_list_Misalign)))


for i in range(Rank):
    
    index         = int(P_value_Rank.iloc[i,0])
    Normal[i,:]   = Normal_FeatureData.iloc[index,:].values
    Misalign[i,:]   = Misalign_FeatureData.iloc[index,:].values
# 정상, 고장 특징값 합치기    
FeatureSelected = pd.DataFrame(np.concatenate([Normal, Misalign] , axis=1))

print("Selected Feature Data Size :", FeatureSelected.shape)

Selected Feature Data Size : (30, 1010)


In [26]:
path = './ProcessedData/[Real]FeatureSelected'   # path = '파일 경로//저장할 파일 이름'
FeatureSelected.to_csv(path, sep=',', header=None, index=None)

In [27]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [28]:
# P-value 낮은 순위
FeatureRank = 1

# PDF 그래프
sb.distplot(Normal_FeatureData.iloc[int(P_value_Rank.iloc[FeatureRank-1,0]),:], label = 'Normal')
sb.distplot(Misalign_FeatureData.iloc[int(P_value_Rank.iloc[FeatureRank-1,0])  ,:], label = 'Misalign')
plt.legend(loc='upper right')
plt.show()

C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\SDML\anaconda3\envs\gan1\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
